# wordslab-notebooks-lib.notebook

> Access wordslab-notebooks Jupyterlab extension version, current notebook path, json content and cell id, and create or update cells.

In [1]:
#| default_exp notebook

In [2]:
#| export
from ast import literal_eval
from html import escape
from inspect import currentframe, getattr_static, getdoc, isfunction, ismethod, signature
from textwrap import dedent
from types import ModuleType, FunctionType, MethodType, BuiltinFunctionType
import typing

from IPython.core.getipython import get_ipython
from IPython.core.interactiveshell import InteractiveShell
from IPython.core.oinspect import Inspector
from IPython.display import display, HTML
import nbformat

from fastcore.utils import patch
from toolslm.funccall import get_schema

## Work together with AI in a Jupyterlab notebook - the Solveit method

A Jupyter notebook is a convenient way to build context for a LLM one cell after the other: you are working in a fully editable conversation, while interacting with AI and with code.

**Jeremy Howard** and his team at **Answer.ai** explored how to work efficiently in this kind of conversation: they developed a method and platform called Solveit.

https://solve.it.com/

We would like to replicate this approach to working with AI in a wordslab notebooks environment. 

Here is how we chose to do it:
- in a jupyterlab notebook, there are two types of cells: markdown and code
- we want to simulate a third type of cell: a "prompt" cell
- the content of this cell is a prompt (text in markdown format) which is sent to an llm when the cell is executed, along with the text of all the cells situated above in the notebook (context)
- the llm response is streamed just below and formatted as markdown.

To simulate this "prompt" cell we need to develop a **Jupyterlab frontend extension** which implements the following behaviors :
- three buttons are added to the cell toolbar: "note", "prompt", "code"
- a click on one of these buttons changes the type of the cell
  - "note" selects a classic markdown cell
  - "prompt" selects a code cell, modified with the special "prompt behavior" defined below
  - "code" selects a classic code cell
- a "prompt" cell is distinguished from a regular code cell by a metadata property registered in the ipynb file
- each cell type is visualized by a specific color in the left border of the cell
  - "note" cell has a green border
  - "prompt" cell a red border
  - "code" cell has a blue border
- the "prompt" cell is a code cell with the specific modified behaviors
  - the code syntax highlighting is replaced by markdown syntax highlighting when the user types text in this cell
  - when the user executes this cell, the frontend extension does the following
    - calls the kernel to inject the following variables
      - __notebook_path with the path and name of the notebook in the workspace
      - __notebook_content with the full json representation of the notebook
      - __cell_id with the id of the current cell
    - then calls the kernel to execute a specific chat(message) python function
      - where the message parameter is the content of the cell
      - and the content of the notebook above the current cell is inluded as context
  - the python chat() function streams the response tokens from the llm to the output section of the code cell, with markdown rendering

See the section "Develop a Jupyterlab frontend extension" at the bottom of this page to understand how the extension was developed.

## Install the Jupyterlab extension - wordslab-notebooks-lib

If you want to use "prompt" cells, you will first need to install the Jupyterlab frontend extension:
- activate your Jupyterlab python virtual environment
- **pip install wordslab-notebooks-lib**
- restart your Jupyterlab server

The extension is **already pre-installed in the wordslab-notebooks environment**.

To be clear: the wordslab-notebooks-lib package contains both: the Javascript Jupyterlab frontend extension AND the python library wich is loaded in the python kernel.

The Jupyterlab frontend extension is reloaded and re-initialized each time you refresh your browser page: 
- to check is the extension is installed and running, look at the browser console and llok for the message 'Wordslab notebooks extension vx.y.z activated'
- hit the refresh button if you encounter a bug and the extension stops working

## Extend the ipython kernel with useful utilities

You also need to install the wordslab-notebooks-lib library in the virtual environnement used by the ipython kernel which runs each notebook in which you want to use the Solveit method.

It is the client of the Jupyterlab extension, and provides many utilties and tools which support this new way of working with AI.

The main python object used to interact with an ipython kernel is the InteractiveShell. You get an instance of it with the get_ipython() method.

In [3]:
shell = get_ipython()
type(shell)

ipykernel.zmqshell.ZMQInteractiveShell

We will start by extending this shell with capabilities useful to work in a notebook with the Solveit method. These extensions are inspired by the library **ipykernel_helper** from **Answer.ai**. As of december 2025, this library is not open source, but it is available to users in the solve.it.com environment and is a dependency of other Apache 2.0 libraries, so I think it is OK to use it as an inspiration.

In [4]:
#| export
@patch
def _get_info(self: Inspector, obj, oname='', formatter=None, info=None, detail_level=0, omit_sections=()):
    """Custom Python formatter for ?? output:
    - suppress the useless code indentation
    - wrap the code in a markdown python code block
    - display the output as markdown
    """
    orig = self._orig__get_info(obj, oname=oname, formatter=formatter, info=info,
                                detail_level=detail_level, omit_sections=omit_sections)
    if detail_level == 0:
        return orig
    info_dict = self.info(obj, oname=oname, info=info, detail_level=detail_level)
    out = []
    if c := info_dict.get('source'): 
        out.append(f"\n```python\n{dedent(c)}\n```")
    if c := info_dict.get('file'): 
        out.append(f"**File:** `{c}`")
    return {'text/markdown': '\n\n'.join(out), 'text/html': '', 'text/plain': orig['text/plain']}

In [5]:
escape??


```python
def escape(s, quote=True):
    """
    Replace special characters "&", "<" and ">" to HTML-safe sequences.
    If the optional flag quote is true (the default), the quotation mark
    characters, both double quote (") and single quote (') characters are also
    translated.
    """
    s = s.replace("&", "&amp;") # Must be done first!
    s = s.replace("<", "&lt;")
    s = s.replace(">", "&gt;")
    if quote:
        s = s.replace('"', "&quot;")
        s = s.replace('\'', "&#x27;")
    return s
```

**File:** `/home/python/cpython-3.12.12-linux-x86_64-gnu/lib/python3.12/html/__init__.py`

In [6]:
#| export
def _safe_str(obj, max_str_len=200):
    "Safely get the string representation of an object, truncating if it exceeds max_len."
    try:
        s = str(obj)
        return s[:max_str_len] + ("…" if len(s) > max_str_len else "")
    except Exception as e: 
        return f"<str error: {str(e)}>"


@patch
def user_items(self: InteractiveShell, max_str_len=200, xtra_ignore=()):
    """Get an overview of the variables & functions defined by the user so far in the notebook.
    The value addded by this function is to filter out all internal ipython and wordslab variables.
    Returns a tuple of dictionaries (user_variables, user_functions):
    - the keys are the variables or function names
    - the value is a truncated string representation of the variable value or the function signature
    The `max_str_len` parameter is used to truncate the string representation of the variables.
    The `xtra_ignore` parameter is used to hide additional names from the result. 
    """
    ns, nsh = self.user_ns, self.user_ns_hidden
    ignore = set()  # Add here the wordslab specific vars and funcs we want to hide
    ignore.add(xtra_ignore)
    rm_types = (
        type, FunctionType, ModuleType, MethodType, BuiltinFunctionType,
        getattr(typing, '_SpecialGenericAlias', ()),
        getattr(typing, '_GenericAlias', ()),
        getattr(typing, '_SpecialForm', ())
    )
    user_items = {k: v for k, v in ns.items()
                  if k not in ignore and k not in nsh}
    user_vars = {k: _safe_str(v, max_str_len=max_str_len)
                 for k, v in user_items.items() if not k.startswith('_') and not isinstance(v, rm_types)}
    user_fns = {k: str(signature(v)) for k, v in user_items.items()
                if isinstance(v, FunctionType) and v.__module__ == '__main__' and not k.startswith('__')}
    return user_vars, user_fns

In [7]:
variables, functions = shell.user_items()
variables, functions

({'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object at 0x72adec178380>',
  'user_items': 'None'},
 {'_safe_str': '(obj, max_str_len=200)'})

In [8]:
#| export
@patch
def get_variables_values(self: InteractiveShell, var_names: list, literal=True):
    """Get a safe and serializable representation of variables values from the user namespace.
    This method preserves real Python values when they are safe literals, otherwise it falls back to strings.
    You can call it in two modes:
    - literal = True : Preserve actual Python values when safe, best for internal tools
    - literal = False : Force everything to strings, best for logging / UI display / debug output
    """
    ns = self.user_ns

    def _maybe_eval(o):
        try:
            literal_eval(repr(o))
            return o
        except:
            return str(o)
    return {v: _maybe_eval(ns[v]) if literal else str(ns[v]) for v in var_names if v in ns}

In [26]:
shell.get_variables_values(var_names=["variables", "functions"])

{'variables': {'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object at 0x72adec178380>',
  'user_items': 'None'},
 'functions': {'_safe_str': '(obj, max_str_len=200)'}}

In [10]:
#| export
def _get_schema(ns: dict, t):
    "Check if tool `t` has errors."
    if t not in ns:
        return f"`{t}` not found. Did you run it?"
    try:
        return {'type': 'function', 'function': get_schema(ns[t], pname='parameters')}
    except Exception as e:
        return f"`{t}`: {e}."

@patch
def get_tools_schemas(self: InteractiveShell, func_names: list):
    """Get a json schema of functions defined in the user namespace which can be used as tools."""
    ns = self.user_ns
    return {f: _get_schema(ns, f) for f in func_names}

In [11]:
# Example tool definition
def example_sum(
    a: int,  # First thing to sum
    b: int = 1,  # Second thing to sum
) -> int:  # The sum of the inputs
    "Adds a + b."
    return a + b

In [28]:
shell.get_tools_schemas(["example_sum"])

{'example_sum': {'type': 'function',
  'function': {'name': 'example_sum',
   'description': 'Adds a + b.\n\nReturns:\n- type: integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First thing to sum'},
     'b': {'type': 'integer',
      'description': 'Second thing to sum',
      'default': 1}},
    'required': ['a']}}}}

## Access the notebook path, cells content and current cell id

The 4 variables below are **silently injected by the Jupyterlab frontend extension** before each code cell is executed.

This will not work if the wordslab-notebooks-lib Jupyterlab extension is not installed.

In [13]:
#| export
def _find_frame_dict(var: str):
    "Find the dict (globals or locals) containing var"
    frame = currentframe().f_back.f_back
    while frame:
        if var in frame.f_globals:
            return frame.f_globals
        frame = frame.f_back
    raise ValueError(f"Could not find {var} in any scope")

def find_var(var: str):
    "Search for var in all frames of the call stack"
    return _find_frame_dict(var)[var]

In [14]:
#| export
class JupyterlabNotebook:
    """Jupyter notebook introspection and metaprogramming."""

    def __init__(self):
        try:
            self.jupyterlab_extension_version
        except:
            raise RuntimeError(
                "The JupyterLab extension for wordslab-notebooks is not activated: "
                "please execute `pip install wordslab-notebooks-lib` in JupyterLab virtual environment "
                "and refresh your web browser."
            )

    @property
    def jupyterlab_extension_version(self):
        """wordslab-notebooks-lib version number injected by the Jupyterlab frontend extension"""
        return find_var("__wordslab_extension_version")

    @property
    def path(self):
        """Relative path of the notebook .ipynb file in the notebook workspace"""
        return find_var("__notebook_path")

    @property
    def content(self):
        """Full content of the notebook returned as a NotebookNode object from the nbformat library"""
        return nbformat.from_dict(find_var("__notebook_content"))

    @property
    def cell_id(self):
        """Unique ID of the current notebook cell, useful to locate the current cell in the full notebook content"""
        return find_var("__cell_id")

In [15]:
notebook = JupyterlabNotebook()

In [16]:
notebook.jupyterlab_extension_version

'0.0.11'

In [17]:
notebook.path

'wordslab-notebooks-lib/nbs/02_notebook.ipynb'

In [18]:
notebook.content.metadata

{'kernelspec': {'display_name': 'wordslab-notebooks-lib',
  'language': 'python',
  'name': 'wordslab-notebooks-lib'},
 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
  'file_extension': '.py',
  'mimetype': 'text/x-python',
  'name': 'python',
  'nbconvert_exporter': 'python',
  'pygments_lexer': 'ipython3',
  'version': '3.12.12'}}

In [19]:
notebook.cell_id

'd16ad869-d651-40bd-af2c-623d82b4edf0'

In [20]:
notebook.cell_id

'59347d6e-d1f7-4d23-b041-143e42887f6d'

## Create, update, delete and run notebook messages

In [ ]:
#| export
def add_cell(
    content: str,  # Content of the cell (i.e the prompt, code, or note cell text)
    placement: str = 'add_after',  # Can be 'add_after', 'add_before', 'at_start', 'at_end'
    cell_id: str = None,  # id of the cell that placement is relative to (if None, uses current cell)
    cell_type: str = 'note',  # Cell type, can be 'code', 'note', or 'prompt'
    notebook_path: str = ''  # Notebook to update, defaults to current notebook
):
    "... to do ..."
    _diff_dialog(placement not in ('at_start','at_end') and not cell_id, "`cell_id` or `placement='at_end'`/`placement='at_start'` must be provided when target notebook is different")
    if placement not in ('at_start','at_end') and not msgid: msgid = find_msg_id()
    ... to do ...
    return new_cell_id

In [ ]:
#| export
def update_cell(
    cell_id: str = None,  # id of the cell to update (if None, uses current cell)
    cell_type: str = None,  # Cell type, can be 'code', 'note', or 'prompt', if None the cell type is preserved
    content: str = None,  # Content of the cell (i.e the prompt, code, or note cell text)
    notebook_path: str = ''  # Notebook to update, defaults to current notebook
):
    """Update an existing cell.
    - Use `content` param to update contents.
    - Only include parameters to update--missing ones will be left unchanged."""
    ... to do ...
    return res

In [ ]:
#| export
def del_cell(
    cell_id: str = None,  # id of cell to delete
    notebook_path: str = ''  # Notebook to update, defaults to current notebook
):
    "Delete a cell from the notebook."
    ... to do ...
    return cell_id

In [ ]:
#| export
def run_cell(
    cell_id: str = None,  # id of cell to execute
    notebook_path: str = ''  # Notebook in which to execute cells, defaults to current notebook
):
    "Adds a cell to the run queue. Use read_cell to see the output once it runs."
    ... to do ...
    return cell_id

## Explore the notebook variables, functions and objects

In [21]:
#| export
@patch
def display_variables_and_functions(self: JupyterlabNotebook):
    """Display the variables and functions defined by the user so far in the notebook."""
    variables, functions = get_ipython().user_items()
    output = "<h4>Variables</h4><table><tr><th>Name</th><th>Value</th></tr>"
    for var, value in variables.items():
        output += f"<tr><td>{var}</td><td>{escape(value)}</td></tr>"
    output += "</table>"
    output += "<h4>Functions</h4><table><tr><th>Name</th><th>Signature</th></tr>"
    for func, sig in functions.items():
        output += f"<tr><td>{func}</td><td>{escape(sig)}</td></tr>"
    output += "</table>"
    display(HTML(output))

In [22]:
notebook.display_variables_and_functions()

Name,Value
shell,<ipykernel.zmqshell.ZMQInteractiveShell object at 0x72adec178380>
user_items,None
variables,"{'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object at 0x72adec178380>', 'user_items': 'None'}"
functions,"{'_safe_str': '(obj, max_str_len=200)'}"
get_variables_values,None
get_tools_schemas,None
notebook,<__main__.JupyterlabNotebook object at 0x72adcdf73dd0>
display_variables_and_functions,None
Name,Signature
_safe_str,"(obj, max_str_len=200)"


In [23]:
#| export
def _safe_getattr(obj, name):
    try:
        return getattr(obj, name)
    except:
        return None

def _safe_attr_doc(obj, name, max_str_len=200):
    # Try to get the static attribute from the class
    try:
        class_attr = getattr_static(obj.__class__, name)
    except AttributeError:
        class_attr = None
    # 1. Property
    if isinstance(class_attr, property):
        attr_doc = getdoc(class_attr)
    # 2. classmethod / staticmethod
    elif isinstance(class_attr, (classmethod, staticmethod)):
        attr_doc = getdoc(class_attr.__func__)
    # 3. Function (instance method)
    elif isfunction(class_attr):
        attr_doc = getdoc(class_attr)
    # 4. Descriptor with __doc__ (other descriptors)
    elif hasattr(class_attr, "__doc__") and class_attr.__doc__:
        attr_doc = getdoc(class_attr)
    # 5. Fallback to instance attribute
    elif hasattr(obj, name):
        instance_attr = getattr(obj, name)
        if not type(instance_attr).__module__ == "builtins" and hasattr(instance_attr, "__doc__") and instance_attr.__doc__:
            attr_doc = getdoc(instance_attr)
        else:
            attr_doc = ""
    # Max length
    if attr_doc is None:
        attr_doc = ""
    elif len(attr_doc) > max_str_len:
        attr_doc = attr_doc[:max_str_len] + "…"
    return attr_doc

@patch
def display_object_members(self: JupyterlabNotebook, obj):
    """Display the attributes and methods of a given python object"""
    obj_class = obj.__class__
    output = (f"<h3>Object of type: {obj_class.__name__}</h3>") 
    output += f"<pre>{getdoc(obj)}</pre>"

    obj_members = [(name, _safe_getattr(obj, name)) for name in dir(obj) if not name.startswith("_")]

    output += "<h4>Attributes</h4>"
    output += "<table><tr><th>Name</th><th>Type</th><th>Value</th><th>Doc</th></tr>"
    for name, value in obj_members:
        if name.startswith("_"):
            continue
        # Skip callables (handled as methods below)
        if callable(value):
            continue
        attr_type = type(value).__name__
        attr_value = _safe_str(value)
        attr_doc = _safe_attr_doc(obj, name)
        output += f"<tr><td>{name}</td><td>{escape(attr_type)}</td><td>{escape(attr_value)}</td><td>{escape(attr_doc)}</td></tr>"
    output += "</table>"

    output += "<h4>Methods</h4>"
    output += "<table><tr><th>Name</th><th>Signatue</th><th>Type</th><th>Doc</th></tr>"
    for name, value in obj_members:
        if name.startswith("_"):
            continue
        # Skip attributes (handled above)
        if not callable(value):
            continue
        # Determine method type
        method_type = "instance method"
        if isfunction(value):
            # Defined on the class
            class_attr = getattr(obj_class, name, None)
            if isinstance(class_attr, classmethod):
                method_type = "class method"
            elif isinstance(class_attr, staticmethod):
                method_type = "static method"
        elif ismethod(value):
            method_type = "instance method"
        # Signature
        try:
            sig = str(signature(value))
        except (ValueError, TypeError):
            sig = "(...)"
        # Doc
        method_doc = _safe_attr_doc(obj, name)
        output += f"<tr><td>{name}</td><td>{escape(sig)}</td><td>{method_type}</td><td>{escape(method_doc)}</td></tr>"
    output += "</table>"

    display(HTML(output))

In [24]:
notebook.display_object_members(notebook)

Name,Type,Value,Doc
cell_id,str,b602a192-b7e7-4976-a26d-4fa1854879b4,"Unique ID of the current notebook cell, useful to locate the current cell in the full notebook content"
content,NotebookNode,"{'metadata': {'kernelspec': {'display_name': 'wordslab-notebooks-lib', 'language': 'python', 'name': 'wordslab-notebooks-lib'}, 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3}, …",Full content of the notebook returned as a NotebookNode object from the nbformat library
jupyterlab_extension_version,str,0.0.11,wordslab-notebooks-lib version number injected by the Jupyterlab frontend extension
path,str,wordslab-notebooks-lib/nbs/02_notebook.ipynb,Relative path of the notebook .ipynb file in the notebook workspace
Name,Signatue,Type,Doc
display_object_members,(obj),instance method,Display the attributes and methods of a given python object
display_variables_and_functions,(),instance method,Display the variables and functions defined by the user so far in the notebook.


## Get $variable values and &tool schemas referenced in prompt cells

In [25]:
#| export
@patch
def get_variables_values(self: JupyterlabNotebook, var_names: list):
    """Get a safe and serializable representation of variables values."""
    return get_ipython().get_variables_values(var_names=var_names)

@patch
def get_tools_schemas(self: JupyterlabNotebook, func_names: list):
    """Get a json schema of functions which can be used as tools."""
    return get_ipython().get_tools_schemas(func_names=func_names)

In [27]:
notebook.get_variables_values(var_names=["variables", "functions"])

{'variables': {'shell': '<ipykernel.zmqshell.ZMQInteractiveShell object at 0x72adec178380>',
  'user_items': 'None'},
 'functions': {'_safe_str': '(obj, max_str_len=200)'}}

In [29]:
notebook.get_tools_schemas(["example_sum"])

{'example_sum': {'type': 'function',
  'function': {'name': 'example_sum',
   'description': 'Adds a + b.\n\nReturns:\n- type: integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First thing to sum'},
     'b': {'type': 'integer',
      'description': 'Second thing to sum',
      'default': 1}},
    'required': ['a']}}}}

In [20]:
#| hide
import nbdev; nbdev.nbdev_export()

## Develop a Jupyterlab frontend extension

### Understand Jupyterlab kernels and frontend extensions

Jupyter kernels technical implementation details

https://chatgpt.com/share/692bea08-4510-8004-b9ab-c02feeb97c08

Jupyterlab extension development tutorial

https://jupyterlab.readthedocs.io/en/latest/extension/extension_tutorial.html

### Intialize the components of a frontend extension

The source code of the Jupyterlab frontend extension can be found in the following files:

Typescript source code, dependencies, and compilation config:

- `src/index.ts`
- `package.json`
- `tsconfig.json`
- `.yarnrc.yml`

Extension manifest and Javascript compiled code

- wordslab_notebooks_lib/labextension
  - package.json
  - static/remoteEntry.97d57e417eaf8ebadeb6.js 

This is how the extension files are included in the python package:

- `MANIFEST.in` 

```
include install.json
include package.json
recursive-include wordslab_notebooks_lib/labextension *

graft wordslab_notebooks_lib/labextension
graft src
```

This is how the extension files are installed in Jupyterlab extensions directory when the python package is installed:

- `pyproject.toml`

```toml
[tool.setuptools]
include-package-data = true 

[tool.setuptools.data-files]
"share/jupyter/labextensions/wordslab-notebooks-lib" = [
  "wordslab_notebooks_lib/labextension/package.json",
  "install.json"
]
"share/jupyter/labextensions/wordslab-notebooks-lib/static" = [
  "wordslab_notebooks_lib/labextension/static/*"
]
```

This how the command `jupyter labextension develop` finds the directory where the extension files live:

- `wordslab_notebooks_lib\__init__.py`

```python
def _jupyter_labextension_paths():
    return [{
        "src": "labextension",
        "dest": "wordslab-notebooks-lib"
    }]
```

This is how the python package is identified as a Jupyterlab extension in pypi:

- `pyproject.toml`

```
classifiers = [ "Framework :: Jupyter :: JupyterLab :: Extensions :: Prebuilt" ]
```

### Install the Jupyterlab frontend extension in development mode

Open a Terminal

```bash
cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib
source $JUPYTERLAB_ENV/.venv/bin/activate

# Install Javascript dependencies
jlpm install

# Build TypeScript extension
jlpm build

# Register the extension with JupyterLab during development
# jupyter labextension develop . --overwrite
rm $JUPYTERLAB_ENV/.venv/share/jupyter/labextensions/wordslab-notebooks-lib
ln -s $WORDSLAB_WORKSPACE/wordslab-notebooks-lib/wordslab_notebooks_lib/labextension/ $JUPYTERLAB_ENV/.venv/share/jupyter/labextensions/wordslab-notebooks-lib

# Verify extension is found
jupyter labextension list
```

### Test the Jupyterlab frontend extension 

After installing the extension in development mode once, you can iterate fast:
- update the code in `src/index.ts`
- build the extension with `jlpm build`

```bash
cd $WORDSLAB_WORKSPACE/wordslab-notebooks-lib
source $JUPYTERLAB_ENV/.venv/bin/activate

# Build TypeScript extension
jlpm build
```
- **refresh** the Jupyterlab single page app in your browser
- test the updated extension

No need to reinstall the extension or to restart Jupyterlab itself, just refresh your browser page.

## Explore the notebook format

https://nbformat.readthedocs.io/en/latest/format_description.html

In [45]:
nb = nbformat.from_dict(__notebook_content)

code_language = nb.metadata.language_info.name
print("> " + code_language + " notebook")

for cell in nb.cells:
    if cell.id == __cell_id: break
        
    is_markdown = cell.cell_type == "markdown"
    is_code = cell.cell_type == "code"
    is_raw = cell.cell_type == "raw"

    print("---------------------")
    print("cell", cell.id, cell.cell_type)
    print("---------------------")
    if is_markdown:
        print(cell.source[:100])
    elif is_code:
        print(f"```{code_language}\n" + cell.source[:100] + "\n```")
    elif is_raw:
        print(cell.source[:100])
    if is_code and cell.execution_count>0 and len(cell.outputs)>0:
        print("---------------------")
        print("cell outputs", cell.id, cell.execution_count)
        print("---------------------")
        for output in cell.outputs:
            if output.output_type == "stream":
                print(f"<{output.name}>")
                print(output.text[:100])
                print(f"</{output.name}>")
            elif output.output_type == "display_data":
                print("<display>")
                if "data" in output:
                    print("  <data>")
                    repr(output.data)
                    print("  </data>")
                if "metadata" in output and len(output.metadata)>0:
                    print("  <metadata>")
                    repr(output.metadata)
                    print("  </metadata>")
                print("</display>")
            elif output.output_type == "execute_result":
                print("<result>")
                if "data" in output:
                    print("  <data>")
                    print(output.data)
                    print("  </data>")
                if "metadata" in output and len(output.metadata)>0:
                    print("  <metadata>")
                    print(output.metadata)
                    print("  </metadata>")
                print("</result>")
            elif output.output_type == "error":
                print("<error>")
                print(output.ename)
                print(output.evalue)
                for frame in output.traceback:
                    print(frame)
                print("</error>")
        print("---------------------")

> python notebook
---------------------
cell 9d8a6aa0-8f58-4860-bcc1-2bfbdcb438b6 markdown
---------------------
# wordslab-notebooks-lib.jupyterlab

> Access wordslab-notebooks Jupyterlab extension version, curre
---------------------
cell 68f3493d-c252-4eb4-844b-abbd68ed3a70 markdown
---------------------
## Work together with AI in a Jupyterlab notebook - the Solveit method

A Jupyter notebook is a conv
---------------------
cell 0ff6fbdc-4a54-4e29-acbb-07529df8cfdd markdown
---------------------
## Install the Jupyterlab extension - wordslab-notebooks-lib

If you want to use "prompt" cells, you
---------------------
cell 65cd4cf8-d77b-4026-b428-bbd9550ea971 markdown
---------------------
## Communicate with the Jupyterlab extension
---------------------
cell ece4d545-8f78-4232-82fb-e837ea0185e4 code
---------------------
```python
#| export
import nbformat
```
---------------------
cell af2f3f45-f0da-4d37-9e39-b37d19ba5650 code
---------------------
```python
class JupyterlabNotebo